In [ ]:
%% capture
!pip install pandas vllm transformers

In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from IPython.display import display
import pandas as pd
import torch
import re

In [ ]:
def extract_boxed(text):
    m = re.search(r'\\boxed\{([^}]*)\}', text)
    return m.group(1) if m else None

def extract_after_think(text: str):
    m = re.search(r'</think>\s*([\s\S]*)', text)
    return m.group(1).strip() if m else None

def extract_answer_xml(text: str):
    m = re.search(r'<final_answer>([\s\S]*?)</final_answer>', text)
    return m.group(1).strip() if m else None

### Configuration

In [ ]:
INPUT_FILE = "test.csv"

In [ ]:
model = "Qwen/Qwen3-32B-AWQ"

max_model_len = 16384
max_token = 16384
gpu_memory_utilization = 0.9
gpu_count = 1

enable_thinking = True

# Recommended for Qwen3 with thinking enabled
temperature = 0.6
top_p = 0.95
top_k = 20
min_p = 0

# # Recommended for Qwen3 with thinking disabled
# temperature = 0.7
# top_p = 0.8
# top_k = 20
# min_p = 0

In [ ]:
SYSTEM_PROMPT = r"""You are a highly specialized AI assistant with expert-level proficiency in financial, compliance, investment, and regulatory examination topics, capable of operating fluently in both Thai and English. Your primary function is to analyze complex queries and deliver responses with the highest degree of accuracy, precision, and relevance.

### Core Objectives

- You must reason logically, utilize your domain expertise, and **ensure that all answers are factually correct**.

- You must **always provide an answer**. A correct answer **always exists**. If uncertain, use logic and context to infer the most likely answer.

- For multiple-choice questions, **output only the letter identifier** (A, B, C, or D). **Do not include the option text.**

- For binary classification questions (e.g., stock prediction), output only the **exact answer string provided in the prompt** (e.g., `Rise`, `Fall`, `ขึ้น`, `ลง`, `เพิ่มขึ้น`, `ลดลง`).

- **Present your final answer enclosed within <final_answer></final_answer> XML tags.**

### Question type

There are two main types of questions:

1. Multiple-choice: **A, B, C, D**. You must output only the letter identifier.
2. Rise or Fall classification: Given the scenario, you must predict whether the closing price of a specific stock (e.g., $INTC) will increase (“Rise”) or decrease (“Fall”). Depending on the question, the choices will be either:
    - "Rise" or "Fall"
    - "ขึ้น" or "ลง"
    - "เพิ่มขึ้น" or "ลดลง"
    
    You must answer only use **exact given word** as your final answer.

### Example

**Example Thai Question 1 (Multiple-choice)**

Question:
```
ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้องแม่นยำคือ A, B, C หรือ D เท่านั้น อย่าอธิบายเยิ่นเย้อหรือให้ข้อมูลเพิ่มเติม
คำถาม: ค่าครองชีพในประเทศที่ใช้ดอลลาร์แพงกว่าประเทศไทยหรือไม่
ตัวเลือกคำตอบ: A: ใช่, B: ไม่, C: ไม่อาจทราบได้, D: ถูกทุกข้อ
คำตอบ:
```
Expected Answer:
```
<final_answer>A</final_answer>
```
---

**Example Thai Question 2 (Rise or Fall classification)**

Question:
```
วิเคราะห์ข้อมูลและทวีตเพื่อสรุปว่าราคาปิดของ $gs จะปรับตัวขึ้นหรือลงในวันที่ 2017-12-20 โปรดยืนยันว่าขึ้นหรือลง

บริบท: วันที่, เปิด, ปิด
Jan 2 2023, 100, 102
Jan 3 2023, 102, 105
Jan 4 2023, 105, 103
Jan 5 2023, 103, 106

2017-12-06: Analysts say $ABC could hit $110 soon. Strong buy recommendation.
```
Expected Answer:
```
<final_answer>ขึ้น</final_answer>
```

---

**Example English Question 1 (Multiple-choice)**

Question:
```
Answer the question with the appropriate options A, B, C and D. Please respond with the exact answer A, B, C or D only. Do not be verbose or provide extra information. 
Question: Which currency is more valuable, the US dollar or the Thai baht?
Answer Choices: A: Thai baht, B: US dollar, C: Both have same value, D: All of that above
Answer:
```
Expected Answer:
```
<final_answer>B</final_answer>
```

---

**Example English Question 2 (Rise or Fall classification)**

Question:
```
Given the data and tweets, could you project whether the closing price of $intc will grow or shrink at 2018-01-23? Please specify either Rise or Fall.

Context: Date, Open, Close
Jan 2 2023, 110, 102
Jan 3 2023, 112, 105
Jan 4 2023, 115, 103
Jan 5 2023, 113, 106

2017-12-06: Analysts say $ABC could hit $100 soon.
```
Expected Answer:
```
<final_answer>Fall</final_answer>
```"""

In [ ]:
# Check
if max_token > max_model_len:
    print(f"Warning: max_token {max_token} is greater than max_model_len {max_model_len}. Setting max_token to max_model_len.")
    max_token = max_model_len

### Initialize

In [ ]:
try:
    dtype = "bfloat16" if torch.cuda.is_bf16_supported() else "float16"
except:
    dtype = "float16"
    
print(f"Dtype: {dtype}")

In [ ]:
df = pd.read_csv(INPUT_FILE)  
display(df)

In [ ]:
# Init LLM
prompts = df["query"].tolist()


sampling_params = SamplingParams(
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    min_p=min_p,
    max_tokens=(max_token)
)

llm = LLM(
    model=model,    
    dtype=dtype,             
    gpu_memory_utilization=gpu_memory_utilization,
    max_model_len=max_model_len,
    tensor_parallel_size=gpu_count
)

tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
# Apply conversations template

system_msg = {"role": "system", "content": SYSTEM_PROMPT}

conversations = [
    [
      system_msg,
      {"role": "user", "content": prompt}
    ]
    for prompt in prompts
]

input_texts = [tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=enable_thinking,  # Set to False to strictly disable thinking
) for messages in conversations]



In [ ]:
print(input_texts[0])

### Generate

In [ ]:
# Generate responses

print(f"Generating {len(prompts)} completions")
outputs = llm.generate(input_texts, sampling_params=sampling_params)

responses = [out.outputs[0].text for out in outputs]

# === 5a. Attach back to original DataFrame ===
# Make sure the order is the same
df["response"] = responses
df["answer"] = df["response"].apply(extract_answer_xml)

display(df)

### Save

In [ ]:
# Save results
OUTPUT_FILE = "output.csv"
SUBMIT_FILE = "submit.csv"

df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
print(f"Saved results to {OUTPUT_FILE}")

df[["id", "answer"]].to_csv(SUBMIT_FILE, index=False, encoding="utf-8")
print(f"Saved submit file to {SUBMIT_FILE}")